In [31]:
Round=145

import warnings
warnings.filterwarnings('ignore')
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.externals import joblib
import os.path
import numerapi

from sklearn.decomposition import PCA
# from MulticoreTSNE import MulticoreTSNE as TSNE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
import random, timeit, string


In [32]:
os.getcwd()

'C:\\Users\\Samit.singh\\Jupyter_Notebooks\\Genetic_Ensemble\\GE_WIP'

In [33]:
napi = numerapi.NumerAPI(verbosity="info")
if napi.check_new_round():
    print("new round has started within the last 24hours!")
else:
    print("no new round within the last 24 hours")

def load_data(ro):
    ros=str(ro)
    if os.path.isfile('../datasets/'+ros+'-train.csv'):
        return pd.read_csv('../datasets/'+ros+'-train.csv'),pd.read_csv('../datasets/'+ros+'-tour.csv')
    else:
        print ('Checking Round')
        cc=napi.get_current_round()
        if cc==ro:
            print ('Downloading files for Round ',ro)
            napi.download_current_dataset(unzip=True)
        else:
            print ('Older round or round not started')


no new round within the last 24 hours


In [34]:
train,tour=load_data(Round)
ship = tour[tour['data_type']=='validation']

In [35]:
# (train.ix[:,-2:])

In [36]:
features=[f for f in list(train) if "feature" in f]
print (features)
targets=[f.split('_')[1] for f in list(train) if "target" in f]
print (targets)
target=targets[0]
target

['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature7', 'feature8', 'feature9', 'feature10', 'feature11', 'feature12', 'feature13', 'feature14', 'feature15', 'feature16', 'feature17', 'feature18', 'feature19', 'feature20', 'feature21', 'feature22', 'feature23', 'feature24', 'feature25', 'feature26', 'feature27', 'feature28', 'feature29', 'feature30', 'feature31', 'feature32', 'feature33', 'feature34', 'feature35', 'feature36', 'feature37', 'feature38', 'feature39', 'feature40', 'feature41', 'feature42', 'feature43', 'feature44', 'feature45', 'feature46', 'feature47', 'feature48', 'feature49', 'feature50']
['bernie', 'elizabeth', 'jordan', 'ken', 'charles', 'frank', 'hillary']


'bernie'

In [37]:
# from sklearn.manifold import TSNE


In [23]:
all_data = np.concatenate(train[features].values, tour[features].values,axis=0)

def get_tsne_features(all_data):
    pca = PCA(n_components=9)
    data_pca = pca.fit_transform(all_data)
    tsne = TSNE(n_jobs=8)
    data_tsne =  TSNE(n_components=2).fit_transform(data_pca)

(502732, 9)

In [ ]:
# Add TSNE features calculated to train and tour data
train["TSNE_1"] = data_tsne[:train.shape[0],0]
train["TSNE_2"] = data_tsne[:train.shape[0],1]
tour["TSNE_1"] = data_tsne[train.shape[0]:train.shape[0]+tour.shape[0],0]
tour["TSNE_2"] = data_tsne[train.shape[0]:train.shape[0]+tour.shape[0],1]

In [24]:
tsne = TSNE(n_jobs=8)
X_embedded = tsne.fit_transform(X)

In [ ]:
X_embedded = TSNE(n_components=2).fit_transform(train[features].values)

In [29]:
X_embedded[0,:]

array([-9.36552894, -1.786815  ])

In [2]:
def PCA_transformed(n):
    pca = PCA(n_components=n)
    pca.fit(train[features])
    return pca

In [7]:
def rounder(x,level=3):
    if x>=.7:
        return .7
    if x<.3:
        return .3
    mu=10**level
    x=x*mu
    if x>(mu/2):
        return (int(x)+1)/float(mu)
    return int(x)/float(mu)
ron = np.vectorize(rounder)

def loss(y,yp):
    return metrics.log_loss(y,ron(yp))

def exo(cut,pca = 24,onTrain=0):
    pca = PCA_transformed(pca)
    X=pca.transform(cut[features])
    X = np.concatenate([X,cut[["TSNE_1","TSNE_2"]]],axis=1)     # adding TSNE features
    if onTrain==2:
        return X
    y=cut['target_'+target]
    if onTrain:
        return X,y
    yp=cut['pred']
    return X,y,yp,loss(y,yp)

# Caluclates consistency score and log loss
def eraLoss(frame,pred,pca):
    
    eras=frame['era'].unique()
    frame['pred']=pred
    li=[]
    for era in eras:
        X,y,yp,l=exo(frame[frame['era']==era], pca=pca)
        # X = np.concatenate(X,tsne_features)
        li.append(l)
    li=np.array(li)
    X,y,yp,l=exo(frame,pca)
    c=len(li[li<.693])*100/len(li)
    print ('Loss:', l, '- at ',c,'%')
    print ('-- -- '*8)
    print()
    return l, c

In [8]:
def rr(x,y):
    if type(x)==type(1) and type(y)==type(1):
        x=int(x)
        y=int(y)
        return random.randint(x,y+1)
    return random.uniform(x,y)

def rrr(a,b): #test b4 use
    if type(a) == int and type(b) == int:
        c= int((b-a)/4)
    else:
        c= (b-a)/4
    if random.choice([0,1]):
        return rr(a,a+c)                   # search in first  quadrant
    return rr(b-c,b)                       # search in fourth quadrant

In [9]:
def param_gen(emap):                       # emap is a dictionary which has range of values [a,b] for each parameter
    m={}
    for i in emap:
        m[i]=rrr(emap[i][0],emap[i][1])    # for each paramter in the dict provide the upper and lower limit values i.e a and b
    return m                               # return a dictionary with single value for each parameter.  

In [10]:
def mutate(a,b):
    ones=[a,b]
    if type(a) == int and type(b) == int:
        muls=[int((a+b)/2),int((2*a+b)/3),int((2*b+a)/3)]
    else:
        muls=[(a+b)/2,(2*a+b)/3,(2*b+a)/3]
#     print (muls)
#     print (ones+muls+muls)
    return random.choice(ones+muls+muls)

def params_child(p1,p2):
#     print (p1,p2)
    m={}
    for i in p1:                         
        if p1[i]==p2[i]:
            m[i]=p1[i]                          # When parameters are same for both the parents keep them in child else mutate
        else:                                   
            m[i]=mutate(p1[i],p2[i])
    if m==p1:
        return 0
    if m==p2:
        return 0
    return m
def rands(n):
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(n))

In [11]:
def getModel(ty='xgb',params={}):
    if ty=='xgb':
        copy = dict(params)                # making a copy of dict, to delete key safely
        del copy["pca"]
        copy['nthread']=-1
        return XGBClassifier(**copy)
    if ty=='cb':
        copy = dict(params)
        del copy["pca"]
        copy['thread_count']= -1
        copy['verbose']= False
        return CatBoostClassifier(**copy)
    if ty=='lgbm':
        copy = dict(params)
        del copy["pca"]
        return LGBMClassifier(**copy)    

In [12]:
maps={
    'xgb':{'params':{'max_depth':[1,15],'n_estimators':[10,2000],'learning_rate':[.01,.9],'gamma':[0,100], 'min_child_weight':[1,20],
           'max_delta_step':[1,1000], 'subsample':[.05,1.0],'colsample_bytree':[.05,1.0], 
           'colsample_bylevel':[.05,1.0], 'reg_alpha':[0,1000], 'reg_lambda':[0,1000], 'pca': [5,35]},
           'hookups':4,
           'children':3,
           'asuras':2,
           'survivors':5
          },
    'lgbm':{'params':{'num_leaves':[2,80],'max_depth':[1,20],'n_estimators':[10,2000],'learning_rate':[.01,.9],
           'subsample':[.05,1.0],'colsample_bytree':[.05,1.0],'reg_alpha':[0,1000],'reg_lambda':[0,1000],'pca': [5,35]},
           'hookups':10,
           'children':4,
           'asuras':6,
           'survivors':9
          },
    'cb':{'params':{'depth':[1,15],
          'iterations':[250,750],
          'learning_rate':[0.03,0.3], 
          'l2_leaf_reg':[3,100],
          'border_count':[32,200],'pca': [5,35]
          },
          'hookups':4,
           'children':4,
           'asuras':4,
          'survivors':5
         }
}

In [13]:
# this populates the performance of all the classifiers in genome dataframe
def simulate():
    for i,j in genome.iterrows():
        if not j['ran']:
            clock1=timeit.timeit()
            print (j['name'])
            clf=getModel(j['type'],j['params'])
            X,y = exo(train,j['params']["pca"],1)
            Xt,yt=exo(ship,j['params']["pca"],1)
            clf.fit(X,y)
            yp=clf.predict_proba(Xt)[:,1]
            l,c=eraLoss(ship,yp,j['params']["pca"])
            clock2=timeit.timeit()
            genome.loc[i,'log_loss']=l
            genome.loc[i,'consistency']=c
            genome.loc[i,'delay']=round((clock2-clock1)*1000,2)
            genome.loc[i,'ran']=1

In [14]:
def genesis():
    for m in maps:
        for i in range(maps[m]['asuras']):
            genome.loc[len(genome)]=[m+'-gen0-orig-'+rands(4),m,param_gen(maps[m]['params']),.8,0,0,0]

In [15]:
def reproduce(gen):
    for m in maps:
        for h in range(maps[m]['hookups']):
            parents = genome[genome['type']==m].sample(n=2)
            for c in range(maps[m]['children']):
                child=params_child(parents.iloc[0]['params'],parents.iloc[1]['params'])
                if child:
                    genome.loc[len(genome)]=[m+'-gen'+str(gen)+'-h'+str(h)+'-c'+str(c)+'-'+rands(4),m,child,0.8,0,0,0]


In [16]:
def war(genome):
    count=0
    survivors=[]
    for m in maps:
        if count:
            sids=genome[genome['type']==m].sort_values(['consistency','log_loss'], ascending=[False,True]).head(maps[m]['survivors']).index
            survivors=survivors.append(sids)
        else:
            survivors=genome[genome['type']==m].sort_values(['consistency','log_loss'], ascending=[False,True]).head(maps[m]['survivors']).index
        count=count+1
    return genome.iloc[survivors].reset_index(drop=True)

In [17]:
# genome=pd.DataFrame(columns=['name','type','params','log_loss','consistency','delay','ran'])

genome=pd.read_csv('../genome.csv')
genome['params']=genome['params'].apply(eval)

In [3]:
def submission_func(prediction_probabilities, path, target):
    results_df = pd.DataFrame(data=prediction_probabilities,columns=['probability_'+target])
    joined = pd.DataFrame(tour['id']).join(results_df)
    assert(len(joined) == len(tour))
    joined.to_csv(path, index=False)
    
    # Adding public and private keys
    a='6X4PZS5QYKRW5RIN6LK4PY3KGAIQBZJPEW5CPEXNAM6IRBDTDLG4QV3NFWMMJQZI'
    b='WWYHT5NBSLOPKL6VLPNFRZMRG5MHDKPI'
    napi = numerapi.NumerAPI(b, a)
    
    tim={'bernie':1, 'charles':5, 'elizabeth':2, 'jordan':3, 'ken':4, 'frank':6, 'hillary':7}
    
    try:
        submission_id = napi.upload_predictions(path,tournament=tim[target])
        print ("SUBMISSION UPLOADED for {}".format(target))
    except KeyboardInterrupt:
        raise
    except:
        print ("\n Error Uploading submission for {} \n".format(target))    

In [ ]:

for x in targets:
    target=x
    print (target)
    
    for generation in range(2,5):
        reproduce(generation)
        genesis()
        simulate()
        genome=war(genome)
        genome.to_csv('../genome.csv',index=False)
        genome.to_csv('../genome-gen'+str(generation)+'.csv',index=False)

    # Stacking by kfold split


    train_copy = train.copy()
    tour_copy = tour.copy()


    from sklearn.model_selection import GroupKFold

    gkf = GroupKFold(n_splits=5)
    X,Y = exo (train,1,onTrain=1)   # We just need index so PCA -1 does not matter here             
    kfold_split = gkf.split(X, Y, groups=train.era)

    for train_index, test_index in kfold_split:
        print("TRAIN:", train_index, "TEST:", test_index)

        for i,j in genome.iterrows():
            clf=getModel(j['type'],j['params'])
            X,Y = exo(train,j['params']["pca"],1)               # get X (train) as per n_components in PCA
            Xt,yt=exo(tour,j['params']["pca"],1)                # get X (tour) as per n_components in PCA
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
            clf.fit(X_train,y_train)
            train_copy.loc[test_index,"feature_{}_{}".format(j['type'],i)]=clf.predict_proba(X_test)[:,1]

            clf.fit(X,Y)
            tour_copy["feature_{}_{}".format(j['type'],i)]=clf.predict_proba(Xt)[:,1]  


    # Training a stacker
    features_new=[f for f in list(train_copy) if "feature_" in f]
    X_reg = train_copy[features_new]
    y_reg = train_copy['target_'+target]

    # Can add a different stacker here probably neural network!
    stacker = LogisticRegression(max_iter=250).fit(X_reg,y_reg)
    final_preds = stacker.predict_proba(tour_copy[features_new])[:,1]

      
    result_path ='../submissions/'+target+"meta_model_stacked_submission.csv"
    
    submission_func(final_preds,result_path,target)

  

In [ ]:
# stop here

In [29]:
a = np.array([[0,1,2],[1,1,1]])
b = np.array([[3,4],[2,2]])
c = np.concatenate([a,b],axis=1)

In [30]:
c

array([[0, 1, 2, 3, 4],
       [1, 1, 1, 2, 2]])

In [11]:
xx = np.array([[1,2,3,4,5],[1,1,1,1,1]])

In [12]:
xx[-2:]

array([[1, 2, 3, 4, 5],
       [1, 1, 1, 1, 1]])

In [ ]:
tsne_train = tsne_all[:X_train.shape[0]]

In [ ]:
# Concat with tsne features with train data
X = np.concatenate([X, data_tsne[:train.shape[0]]], axis=1)

# Concat with tsne features with validation data
X = np.concatenate([X, data_tsne[:train.shape[0]]], axis=1)